In this notebook, I estimated the stability of the ICRF axes in the frame of ICRF3.

In [24]:
from astropy.table import Table, join, Column
from astropy import units as u
import numpy as np
# My modules
from my_progs.catalog.vsh_deg1_cor import vsh_deg01_fitting
from my_progs.catalog.vsh_deg2_cor import vsh_deg02_fitting, residual_calc02
from my_progs.catalog.write_output import print_vsh1_corr, print_vsh2_corr
from my_progs.catalog.read_icrfn import read_icrf2, read_icrf3
from my_progs.catalog.pos_diff import radio_cat_diff_calc

In [17]:
# Read ICRF2 data
icrf2 = read_icrf2()

# Read ICRF3 S/X catalog
icrf3sx = read_icrf3(wv="sx")

# Read ICRF3 K catalog
icrf3k = read_icrf3(wv="k")

# Read ICRF3 X/Ka catalog
icrf3xka = read_icrf3(wv="xka")

# ICRF3 defining source list
alllist = Table(icrf3sx)
alllist.keep_columns(["iers_name", "type"])
deflist = alllist[alllist["type"] == "D"]

In [20]:
icrf23sx = radio_cat_diff_calc(icrf2, icrf3sx, "iers_name", label=["2", "3sx"])
icrf3k3sx = radio_cat_diff_calc(icrf3k, icrf3sx, "iers_name", label=["3k", "3sx"])
icrf3xka3sx = radio_cat_diff_calc(icrf3xka, icrf3sx, "iers_name", label=["3xka", "3sx"])

In [27]:
# Use only defining sources
icrf23sxdef = join(icrf23sx, deflist, keys="iers_name")
icrf3k3sxdef = join(icrf3k3sx, deflist, keys="iers_name")
icrf3xka3sxdef = join(icrf3xka3sx, deflist, keys="iers_name")


In [28]:
# ICRF2 - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf23sxdef["dra"])
ddec = np.array(icrf23sxdef["ddec"])
dra_err = np.array(icrf23sxdef["dra_err"])
ddec_err = np.array(icrf23sxdef["ddec_err"])
ra_rad = np.array(icrf23sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf23sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf23sxdef["dra_ddec_cov"])

# Transformation parameters
# l_max = 1
w1_all, sig1_all, corrcoef1_all, _, _, _ = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None")

# l_max = 2
w2_all, sig2_all, corrcoef2_all, _, _, _ = vsh_deg02_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3
w2 = w2_all * 1.e3
sig2 = sig2_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]                 "
      "                  Glide [uas]               \n"
      "               x             y             z"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "l_max=1  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "l_max=2  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size, w1[3], sig1[3], w1[4], sig1[4], w1[5], sig1[5],
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2],
       w2[3], sig2[3], w2[4], sig2[4], w2[5], sig2[5],
       w2[0], sig2[0], w2[1], sig2[1], w2[2], sig2[2]))

quad_names = Column(["ER22", "EI22", "ER21", "EI21", "E20",
                     "MR22", "MI22", "MR21", "MI21", "M20"])
t_quad = Table([quad_names, w2[6:], sig2[6:]], names=["Quadrupolar term", "Estimate", "Error"])
t_quad["Estimate"].format = "%5.0f"
t_quad["Error"].format = "%5.0f"
print(t_quad)

print("Correlation coefficient between parameters in 'l_max=1' fit")
print_vsh1_corr(corrcoef1_all, deci_digit=1, included_one=False)

print("Correlation coefficient between parameters in 'l_max=2' fit")
print_vsh2_corr(corrcoef2_all, deci_digit=1, included_one=False)

# apriori statistics (weighted)
#         mean for RA:      0.046 
#         wrms for RA:      0.466 
#          std for RA:      0.463 
#        mean for Dec:      0.205 
#        wrms for Dec:      1.888 
#         std for Dec:      1.873 

# apriori reduced Chi-square for:      1.515
# posteriori statistics of vsh02 fit (weighted)
#         mean for RA:      0.049 
#          rms for RA:      0.462 
#          std for RA:      0.458 
#        mean for Dec:      0.124 
#         rms for Dec:      1.874 
#         std for Dec:      1.866 

# posteriori reduced Chi-square for:      0.819
# goodness-of-fit is      0.999
Estimates (   296 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]                                   Glide [uas]               
               x             y             z               x             y             z
-------------------------------------------------------------------------

In [29]:
# ICRF3 K - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf3k3sxdef["dra"])
ddec = np.array(icrf3k3sxdef["ddec"])
dra_err = np.array(icrf3k3sxdef["dra_err"])
ddec_err = np.array(icrf3k3sxdef["ddec_err"])
ra_rad = np.array(icrf3k3sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf3k3sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf3k3sxdef["dra_ddec_cov"])

# Transformation parameters
# l_max = 1
w1_all, sig1_all, corrcoef1_all, _, _, _ = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None")

# l_max = 2
w2_all, sig2_all, corrcoef2_all, _, _, _ = vsh_deg02_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3
w2 = w2_all * 1.e3
sig2 = sig2_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]                 "
      "                  Glide [uas]               \n"
      "               x             y             z"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "l_max=1  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "l_max=2  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size, w1[3], sig1[3], w1[4], sig1[4], w1[5], sig1[5],
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2],
       w2[3], sig2[3], w2[4], sig2[4], w2[5], sig2[5],
       w2[0], sig2[0], w2[1], sig2[1], w2[2], sig2[2]))

quad_names = Column(["ER22", "EI22", "ER21", "EI21", "E20",
                     "MR22", "MI22", "MR21", "MI21", "M20"])
t_quad = Table([quad_names, w2[6:], sig2[6:]], names=["Quadrupolar term", "Estimate", "Error"])
t_quad["Estimate"].format = "%5.0f"
t_quad["Error"].format = "%5.0f"
print(t_quad)

print("Correlation coefficient between parameters in 'l_max=1' fit")
print_vsh1_corr(corrcoef1_all, deci_digit=1, included_one=False)

print("Correlation coefficient between parameters in 'l_max=2' fit")
print_vsh2_corr(corrcoef2_all, deci_digit=1, included_one=False)

# apriori statistics (weighted)
#         mean for RA:     -0.000 
#         wrms for RA:      0.186 
#          std for RA:      0.186 
#        mean for Dec:      0.048 
#        wrms for Dec:      0.291 
#         std for Dec:      0.287 

# apriori reduced Chi-square for:      1.222
# posteriori statistics of vsh02 fit (weighted)
#         mean for RA:      0.005 
#          rms for RA:      0.175 
#          std for RA:      0.174 
#        mean for Dec:      0.022 
#         rms for Dec:      0.280 
#         std for Dec:      0.278 

# posteriori reduced Chi-square for:      1.072
# goodness-of-fit is      0.163
Estimates (   193 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]                                   Glide [uas]               
               x             y             z               x             y             z
-------------------------------------------------------------------------

In [30]:
# ICRF3 XKa - ICRF3 SX
# Transform columns into np.array
dra = np.array(icrf3xka3sxdef["dra"])
ddec = np.array(icrf3xka3sxdef["ddec"])
dra_err = np.array(icrf3xka3sxdef["dra_err"])
ddec_err = np.array(icrf3xka3sxdef["ddec_err"])
ra_rad = np.array(icrf3xka3sxdef["ra"].to(u.radian))
dec_rad = np.array(icrf3xka3sxdef["dec"].to(u.radian))
dra_ddec_cov = np.array(icrf3xka3sxdef["dra_ddec_cov"])

# Transformation parameters
# l_max = 1
w1_all, sig1_all, corrcoef1_all, _, _, _ = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None")

# l_max = 2
w2_all, sig2_all, corrcoef2_all, _, _, _ = vsh_deg02_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None")

# mas -> uas
w1 = w1_all * 1.e3
sig1 = sig1_all * 1.e3
w2 = w2_all * 1.e3
sig2 = sig2_all * 1.e3


# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]                 "
      "                  Glide [uas]               \n"
      "               x             y             z"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "l_max=1  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "l_max=2  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size, w1[3], sig1[3], w1[4], sig1[4], w1[5], sig1[5],
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2],
       w2[3], sig2[3], w2[4], sig2[4], w2[5], sig2[5],
       w2[0], sig2[0], w2[1], sig2[1], w2[2], sig2[2]))

quad_names = Column(["ER22", "EI22", "ER21", "EI21", "E20",
                     "MR22", "MI22", "MR21", "MI21", "M20"])
t_quad = Table([quad_names, w2[6:], sig2[6:]], names=["Quadrupolar term", "Estimate", "Error"])
t_quad["Estimate"].format = "%5.0f"
t_quad["Error"].format = "%5.0f"
print(t_quad)

print("Correlation coefficient between parameters in 'l_max=1' fit")
print_vsh1_corr(corrcoef1_all, deci_digit=1, included_one=False)

print("Correlation coefficient between parameters in 'l_max=2' fit")
print_vsh2_corr(corrcoef2_all, deci_digit=1, included_one=False)

# apriori statistics (weighted)
#         mean for RA:     -0.003 
#         wrms for RA:      0.239 
#          std for RA:      0.239 
#        mean for Dec:     -0.284 
#        wrms for Dec:      0.398 
#         std for Dec:      0.278 

# apriori reduced Chi-square for:      9.263
# posteriori statistics of vsh02 fit (weighted)
#         mean for RA:      0.011 
#          rms for RA:      0.194 
#          std for RA:      0.193 
#        mean for Dec:     -0.006 
#         rms for Dec:      0.256 
#         std for Dec:      0.255 

# posteriori reduced Chi-square for:      2.120
# goodness-of-fit is      0.000
Estimates (   176 sources)
--------------------------------------------------------------------------------------------
               Rotation [uas]                                   Glide [uas]               
               x             y             z               x             y             z
-------------------------------------------------------------------------